In [1]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow import keras
import librosa

with open('mfcc_data.npy', 'rb') as f:
    mfcc_data = np.load(f)
print(f'MFCC data shape: {mfcc_data.shape}')

label_path = 'deam-dataset/DEAM_Annotations/annotations/annotations averaged per song/song_level/';
labels_1_2000 = pd.read_csv(os.path.join(label_path, 'static_annotations_averaged_songs_1_2000.csv'))
labels_2000_2058 = pd.read_csv(os.path.join(label_path,'static_annotations_averaged_songs_2000_2058.csv'))
labels = pd.concat([labels_1_2000, labels_2000_2058], ignore_index=True, sort=False)
labels = labels[labels_1_2000.columns]
labels.columns = labels.columns.str.replace(' ', '')

valence = labels['valence_mean'].to_numpy()
arousal = labels['arousal_mean'].to_numpy()
y = np.vstack([valence, arousal]).T
print(f'Labels shape: {y.shape}')

/opt/conda/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


MFCC data shape: (1802, 1920, 20)
Labels shape: (1802, 2)


In [2]:
# input: 3D matrix representation of data(data_length, frames, coeffs), time-step size (in seconds)
# output: 4D matrix representation of split into time-steps (data_length, n_time_steps, frames per timestep, coeffs)
def split_timesteps(data, time_step=0.5):
    n_frames = data.shape[1]
    time_step_size = librosa.time_to_frames(time_step, sr=22050, hop_length=512)
    n_time_steps = math.floor(n_frames / time_step_size)
    
    print(f'Total no. of frames per data point: {n_frames}')
    print(f'No. of time steps: {n_time_steps}')
    print(f'No. of frames per time step: {time_step_size}')
    
    new_data = np.empty([data.shape[0], n_time_steps, time_step_size, data.shape[2]])
    
    frame_idxes = np.arange(0, n_frames, time_step_size)[:n_time_steps]
    for data_i in range(data.shape[0]):
        data_pt = np.empty([n_time_steps, time_step_size, data.shape[2]])
        for i, frame_idx in enumerate(frame_idxes):
            start_frame = frame_idx
            end_frame = frame_idx + time_step_size
            if end_frame > n_frames: 
                break
            data_pt[i] = data[data_i][start_frame:end_frame, :]
        new_data[data_i] = data_pt
    return new_data
    

In [3]:
split_mfcc = split_timesteps(mfcc_data, time_step=0.5)
split_mfcc.shape

Total no. of frames per data point: 1920
No. of time steps: 91
No. of frames per time step: 21


(1802, 91, 21, 20)

In [4]:
# from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# idx = np.arange(split_mfcc.shape[0])
# X_trn_idx, X_test_idx, y_trn, y_test = train_test_split(idx, y, test_size=0.2, random_state=23)
# X_train_idx, X_valid_idx, y_train, y_valid = train_test_split(X_trn_idx, y_trn, test_size=0.33, random_state=23)

# X_mfcc_train = split_mfcc[X_train_idx]
# X_mfcc_valid = split_mfcc[X_valid_idx]
# X_mfcc_test = split_mfcc[X_test_idx]

In [5]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

def categorize_val_arousal(data):
    cat_data = []
    # A: active, P: passive
    # P: positive, N: negative
    for point in data:
        valence, arousal = point[0], point[1]
        if valence >= 5 and arousal >= 5:
            cat_data.append('AP')
        elif valence < 5 and arousal >= 5:
            cat_data.append('AN')
        elif valence >= 5 and arousal < 5:
            cat_data.append('PP')
        elif valence < 5 and arousal < 5:
            cat_data.append('PN')
    return np.array(cat_data, dtype='str')

idx = np.arange(split_mfcc.shape[0])

sss_test = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=23)
for train_index, test_index in sss_test.split(idx, categorize_val_arousal(y)):
    X_trn_idx, X_test_idx = idx[train_index], idx[test_index]
    y_trn, y_test = y[train_index], y[test_index]

sss_valid = StratifiedShuffleSplit(n_splits=5, test_size=0.33, random_state=23)
for train_index, test_index in sss_test.split(X_trn_idx, categorize_val_arousal(y_trn)):
    X_train_idx, X_valid_idx = X_trn_idx[train_index], X_trn_idx[test_index]
    y_train, y_valid = y_trn[train_index], y_trn[test_index]

X_mfcc_train = split_mfcc[X_train_idx]
X_mfcc_valid = split_mfcc[X_valid_idx]
X_mfcc_test = split_mfcc[X_test_idx]

In [6]:
from functools import partial
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, 
                                     TimeDistributed, LSTM,
                                     Flatten, Dense, Dropout, RepeatVector, LeakyReLU,
                                     LayerNormalization, BatchNormalization)

leaky_relu = LeakyReLU(alpha=0.05)
conv_layer = partial(Conv1D, kernel_size=4, activation=leaky_relu, padding='SAME', strides=1)
pooling = partial(MaxPooling1D, pool_size=4, strides=4, padding='SAME')

model = keras.models.Sequential([
    TimeDistributed(LayerNormalization()),
    TimeDistributed(conv_layer(input_shape=(1920, 20), filters=32, kernel_size=6)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=64)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=128)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=128)),
    TimeDistributed(pooling()),
    TimeDistributed(Flatten()),
    LSTM(64),
    Dense(128, activation=leaky_relu),
    Dropout(0.3),
    Dense(2)
])

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

optimizer = keras.optimizers.Nadam(lr=0.00002)
model.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])
history = model.fit(X_mfcc_train, y_train, 
                    epochs=100,
                    validation_data=(X_mfcc_valid, y_valid), 
                    batch_size=16, 
                    callbacks=early_stopping)  

Epoch 1/100
72/72 [==============================] - 2s 24ms/step - loss: 4.6696 - mean_squared_error: 23.3587 - val_loss: 4.4189 - val_mean_squared_error: 20.9845
Epoch 2/100
72/72 [==============================] - 1s 16ms/step - loss: 3.9628 - mean_squared_error: 17.3711 - val_loss: 3.3929 - val_mean_squared_error: 12.9684
Epoch 3/100
72/72 [==============================] - 1s 16ms/step - loss: 2.7678 - mean_squared_error: 9.3908 - val_loss: 2.1153 - val_mean_squared_error: 5.8162
Epoch 4/100
72/72 [==============================] - 1s 16ms/step - loss: 1.7408 - mean_squared_error: 4.3344 - val_loss: 1.2895 - val_mean_squared_error: 2.4999
Epoch 5/100
72/72 [==============================] - 1s 16ms/step - loss: 1.2510 - mean_squared_error: 2.3823 - val_loss: 1.0404 - val_mean_squared_error: 1.5788
Epoch 6/100
72/72 [==============================] - 1s 16ms/step - loss: 1.1469 - mean_squared_error: 2.0126 - val_loss: 0.9923 - val_mean_squared_error: 1.4207
Epoch 7/100
72/72 [=====

In [8]:
from sklearn.metrics import accuracy_score, confusion_matrix

score = model.evaluate(X_mfcc_test, y_test)
print(f'Test reg RMSE: {list(map(math.sqrt, score))}')
y_pred = model.predict(X_mfcc_test)

cat_y_pred = categorize_val_arousal(y_pred)
cat_y_test = categorize_val_arousal(y_test)

accuracy = accuracy_score(cat_y_test, cat_y_pred)
print(f'Test Accuracy: {accuracy}')

pd_y_actu = pd.Series(cat_y_test, name='Actual')
pd_y_pred = pd.Series(cat_y_pred, name='Predicted')
df_confusion = pd.crosstab(pd_y_actu, pd_y_pred)
print(f'Confusion matrix: \n{df_confusion}')

12/12 [==============================] - 0s 9ms/step - loss: 0.7464 - mean_squared_error: 0.8855
Test reg RMSE: [0.8639533762118491, 0.9410162153356972]
Test Accuracy: 0.6260387811634349
Confusion matrix: 
Predicted  AN   AP   PN
Actual                 
AN          2   26   18
AP          3  112   15
PN          2   23  112
PP          0   27   21


In [9]:
y_pred[:5]

array([[5.7791815, 5.8042803],
       [3.7000713, 3.385582 ],
       [5.305857 , 5.455228 ],
       [5.6549506, 5.777587 ],
       [4.314077 , 4.204838 ]], dtype=float32)

In [10]:
y_test[:5]

array([[4.2, 6.2],
       [5. , 2.4],
       [5.4, 5.5],
       [5.2, 5.2],
       [4.7, 3.1]])

In [ ]:
# Hyperparameters:
# Time step: 0.5s

final_model = keras.models.Sequential([
    TimeDistributed(LayerNormalization()),
    TimeDistributed(conv_layer(input_shape=(1920, 20), filters=32, kernel_size=6)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=64)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=128)),
    TimeDistributed(pooling()),
    TimeDistributed(conv_layer(filters=128)),
    TimeDistributed(pooling()),
    TimeDistributed(Flatten()),
    LSTM(64),
    Dense(128, activation=leaky_relu),
    Dropout(0.3),
    Dense(2)
])

X_final_train = np.concatenate([X_mfcc_train, X_mfcc_valid], axis=0)
y_final_train = np.concatenate([y_train, y_valid], axis=0)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

optimizer = keras.optimizers.Nadam(lr=0.000005)
final_model.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])
history = final_model.fit(X_final_train, y_final_train, 
                    epochs=150,
                    validation_data=(X_mfcc_test, y_test), 
                    batch_size=16, 
                    callbacks=early_stopping)  

Epoch 1/150
91/91 [==============================] - 2s 21ms/step - loss: 4.7713 - mean_squared_error: 24.2772 - val_loss: 4.6947 - val_mean_squared_error: 23.5338
Epoch 2/150
91/91 [==============================] - 1s 15ms/step - loss: 4.6420 - mean_squared_error: 23.0738 - val_loss: 4.5447 - val_mean_squared_error: 22.1486
Epoch 3/150
91/91 [==============================] - 1s 15ms/step - loss: 4.4627 - mean_squared_error: 21.4484 - val_loss: 4.3384 - val_mean_squared_error: 20.3195
Epoch 4/150
91/91 [==============================] - 1s 15ms/step - loss: 4.2281 - mean_squared_error: 19.4292 - val_loss: 4.0747 - val_mean_squared_error: 18.1081
Epoch 5/150
91/91 [==============================] - 1s 15ms/step - loss: 3.9354 - mean_squared_error: 17.0515 - val_loss: 3.7446 - val_mean_squared_error: 15.5368
Epoch 6/150
91/91 [==============================] - 1s 15ms/step - loss: 3.5764 - mean_squared_error: 14.3961 - val_loss: 3.3683 - val_mean_squared_error: 12.8690
Epoch 7/150
91/9

In [ ]:
score = final_model.evaluate(X_mfcc_test, y_test)
print(f'Final test reg RMSE: {list(map(math.sqrt, score))}')
y_pred = final_model.predict(X_mfcc_test)

cat_y_pred = categorize_val_arousal(y_pred)
cat_y_test = categorize_val_arousal(y_test)

accuracy = accuracy_score(cat_y_test, cat_y_pred)
print(f'Final test Accuracy: {accuracy}')

pd_y_actu = pd.Series(cat_y_test, name='Actual')
pd_y_pred = pd.Series(cat_y_pred, name='Predicted')
df_confusion = pd.crosstab(pd_y_actu, pd_y_pred)
print(f'Final confusion matrix: \n{df_confusion}')